# Sistema de Recomendação Agrícola Inteligente:

## Introdução

Este notebook apresenta a concepção e o detalhamento técnico do sistema de recomendação agrícola desenvolvido para o contexto do Distrito Federal e Entorno. O projeto visa solucionar o desafio de conectar pequenos produtores de olerícolas e frutas a consumidores, com foco na oferta de produtos frescos e locais, alinhados às preferências individuais, e na otimização da logística.

Para alcançar esses objetivos, foi implementada uma arquitetura de recomendação híbrida. Esta abordagem conjuga técnicas de Filtro Colaborativo (CF) e Filtragem Baseada em Conteúdo (CB), enriquecida com filtros geoespaciais e a consideração de preferências explícitas dos usuários.


## 1. Configuração do Ambiente

Primeiro, importamos as bibliotecas necessárias para o processamento de dados, modelagem e cálculos geoespaciais.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from geopy.distance import geodesic
import random
from datetime import datetime
import os
import json 

## 2. Carregamento e Pré-processamento de Dados

Nesta seção, carregamos e preparamos os dois principais datasets:

1.  **`produtos_cooperativas_preenchido_precos_similares.csv`**: Contém informações sobre cooperativas/associações, suas localizações (Latitude, Longitude) e os produtos que comercializam, incluindo seus preços.
2.  **`produtos.csv`**: Detalha características de cada produto, como categoria, subcategoria, perfil de sabor, textura, cor e uso culinário principal. Essas características são cruciais para o modelo baseado em conteúdo.

In [ ]:
# Variáveis globais (simulando o comportamento do módulo real)
df_cooperativas = pd.DataFrame()
df_produtos_features = pd.DataFrame()
df_ratings_global = pd.DataFrame() # Para armazenar todos os ratings
all_available_products = []
product_to_idx = {}
cosine_sim_matrix = None
svd_model = None
user_initial_preferences = {} # user_id: {'categories': ['Fruta', 'Verdura'], 'location': (lat, lon)}

# Nomes dos arquivos de persistência
RATINGS_FILE = 'data/user_ratings.csv'
PREFERENCES_FILE = 'data/user_preferences.json'

def load_and_preprocess_data_notebook():
    global df_cooperativas, df_produtos_features, all_available_products, product_to_idx, cosine_sim_matrix, svd_model, df_ratings_global, user_initial_preferences

    print("Verificando diretório de dados e carregando arquivos base...")
    os.makedirs('data', exist_ok=True)

    try:
        df_cooperativas_raw = pd.read_csv("produtos_cooperativas_preenchido_precos_similares.csv")
        df_produtos_features_raw = pd.read_csv("produtos.csv")
    except FileNotFoundError:
        print("ERRO: Arquivos CSV de base não encontrados. Certifique-se de que 'produtos_cooperativas_preenchido_precos_similares.csv' e 'produtos.csv' estão no local correto.")
        return False

    print("Pré-processando dados das cooperativas...")
    metadata_cols = ['Nome', 'Latitude', 'Longitude', 'Tipo_Organizacao', 'Ano_Fundacao',
                     'Certificacao_Organica_Geral', 'Selo_Agricultura_Familiar_Possui',
                     'Horario_Funcionamento_Atendimento', 'Regioes_Entrega',
                     'Formas_Pagamento_Aceitas', 'Faz_Entrega']
    product_cols_coop = [col for col in df_cooperativas_raw.columns if col not in metadata_cols]

    def clean_price(price_str):
        if isinstance(price_str, (int, float)): return float(price_str)
        if isinstance(price_str, str):
            price_str = price_str.replace(',', '.')
            try: return float(price_str)
            except ValueError: return 0.0
        return 0.0

    for col in product_cols_coop:
        df_cooperativas_raw[col] = df_cooperativas_raw[col].apply(clean_price)

    df_cooperativas_melted = df_cooperativas_raw.melt(
        id_vars=metadata_cols, value_vars=product_cols_coop,
        var_name='Nome_Produto', value_name='Preco'
    )
    df_cooperativas = df_cooperativas_melted[df_cooperativas_melted['Preco'] > 0].copy()
    df_cooperativas.rename(columns={'Nome': 'Nome_Cooperativa'}, inplace=True)
    all_available_products = sorted(df_cooperativas['Nome_Produto'].unique())
    print(f"Dados das cooperativas carregados. Total de {len(df_cooperativas)} ofertas produto-cooperativa.")

    print("Pré-processando dados de características dos produtos...")
    df_produtos_features = df_produtos_features_raw.copy()
    df_produtos_features.set_index('Nome_Produto', inplace=True)
    feature_cols_for_CB = ['Categoria_Produto', 'Subcategoria_Produto', 'Perfil_Sabor_Predominante',
                           'Textura_Predominante', 'Cor_Predominante_Visual', 'Uso_Culinario_Principal']
    
    for col in feature_cols_for_CB:
        if col not in df_produtos_features.columns:
            df_produtos_features[col] = "N/A"

    df_produtos_features['Combined_Features'] = df_produtos_features[feature_cols_for_CB].apply(
        lambda row: ' '.join(row.astype(str).values), axis=1
    )
    
    missing_in_features = [p for p in all_available_products if p not in df_produtos_features.index]
    for p_name in missing_in_features:
        if p_name not in df_produtos_features.index:
            df_produtos_features.loc[p_name, 'Combined_Features'] = "Informacao Indisponivel"
            df_produtos_features.loc[p_name, 'Categoria_Produto'] = "Desconhecida"
    print(f"Dados de características dos produtos carregados. Total de {len(df_produtos_features)} produtos únicos.")

    print("--- Carregando Ratings e Preferências persistidas ---")
    if os.path.exists(RATINGS_FILE):
        try:
            df_ratings_global = pd.read_csv(RATINGS_FILE, parse_dates=['timestamp'])
            print(f"Ratings carregados de {RATINGS_FILE}. Total: {len(df_ratings_global)}")
        except Exception as e:
            print(f"Erro ao carregar ratings de {RATINGS_FILE}: {e}. Iniciando com DataFrame vazio.")
            df_ratings_global = pd.DataFrame()
    else:
        print(f"Arquivo de ratings '{RATINGS_FILE}' não encontrado. Iniciando com DataFrame vazio.")
        df_ratings_global = pd.DataFrame()

    if os.path.exists(PREFERENCES_FILE):
        try:
            with open(PREFERENCES_FILE, 'r') as f:
                loaded_prefs = json.load(f)
                user_initial_preferences = {int(k): {'name': v.get('name', ''), 
                                                     'categories': v.get('categories', []), 
                                                     'location': tuple(v['location']) if isinstance(v.get('location'), list) else None} 
                                            for k, v in loaded_prefs.items()}
            print(f"Preferências de usuário carregadas de {PREFERENCES_FILE}.")
        except Exception as e:
            print(f"Erro ao carregar preferências de usuário de {PREFERENCES_FILE}: {e}. Iniciando com preferências vazias.")
            user_initial_preferences = {}
    else:
        print(f"Arquivo de preferências '{PREFERENCES_FILE}' não encontrado. Iniciando com preferências vazias.")
        user_initial_preferences = {}
            
    # Simular alguns ratings iniciais se df_ratings_global ainda estiver vazio (após tentar carregar)
    if df_ratings_global.empty and all_available_products:
        print("Gerando e simulando alguns ratings iniciais para o modelo SVD para atender ao requisito de 5000 linhas...")
        num_initial_users_simulated = 500 # Aumentado de 10 para 500
        min_ratings_per_user = 10 # Cada usuário simula 10 avaliações
        
        ratings_init_data = []
        for uid_init in range(1, num_initial_users_simulated + 1):
            num_prods_to_rate = min(min_ratings_per_user, len(all_available_products))
            if num_prods_to_rate == 0:
                continue
            prods_to_rate_init = random.sample(all_available_products, num_prods_to_rate)
            
            for prod_init in prods_to_rate_init:
                coop_name = "N/A"
                possible_coops = df_cooperativas[df_cooperativas['Nome_Produto'] == prod_init]['Nome_Cooperativa']
                if not possible_coops.empty:
                    coop_name = possible_coops.sample(1).iloc[0]
                
                ratings_init_data.append({
                    'user_id': uid_init, 
                    'item_id': prod_init, 
                    'rating': random.randint(1, 5), # Avaliações de 1 a 5 (mais realistas)
                    'timestamp': datetime.now(),
                    'cooperative_name': coop_name
                })
        df_ratings_global = pd.DataFrame(ratings_init_data)
        save_ratings_to_csv_notebook() # Usa a função de salvar para o notebook
        print(f"Gerados {len(df_ratings_global)} ratings simulados e salvos em {RATINGS_FILE}.")
    elif df_ratings_global.empty: # Caso não haja produtos para simular
        print("AVISO: Nenhum produto disponível, ratings simulados não gerados.")

    print("Treinando modelo SVD para Filtro Colaborativo...")
    if not df_ratings_global.empty:
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(df_ratings_global[['user_id', 'item_id', 'rating']], reader)
        trainset = data.build_full_trainset()
        svd_model = SVD(n_factors=50, n_epochs=20, random_state=42, biased=True, verbose=False)
        svd_model.fit(trainset)
        print("Modelo SVD treinado com sucesso.")
    else:
        svd_model = None
        print("Modelo SVD não treinado (sem dados de avaliações).")

    print("Computando matriz de similaridade de Cosseno (Content-Based)...")
    if all_available_products and not df_produtos_features.empty:
        valid_products_for_tfidf = [p for p in all_available_products if p in df_produtos_features.index]
        if valid_products_for_tfidf:
            product_features_for_tfidf = df_produtos_features.loc[valid_products_for_tfidf, 'Combined_Features'].fillna("Informacao Indisponivel")
            if not product_features_for_tfidf.empty:
                tfidf_vectorizer = TfidfVectorizer(stop_words=None)
                tfidf_matrix = tfidf_vectorizer.fit_transform(product_features_for_tfidf)
                cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
                product_to_idx = {product_name: i for i, product_name in enumerate(valid_products_for_tfidf)}
                print("Matriz de similaridade de Cosseno computada.")
            else:
                cosine_sim_matrix = None
                product_to_idx = {}
                print("AVISO: Não há dados de features válidos para computar TF-IDF.")
        else:
            cosine_sim_matrix = None
            product_to_idx = {}
            print("AVISO: Nenhum produto em 'all_available_products' encontrado no índice de 'df_produtos_features'.")
    else:
        cosine_sim_matrix = None
        product_to_idx = {}
        print("AVISO: 'all_available_products' ou 'df_produtos_features' estão vazios, TF-IDF não computado.")

    return True

def save_ratings_to_csv_notebook():
    global df_ratings_global
    df_to_save = df_ratings_global.copy()
    df_to_save['timestamp'] = df_to_save['timestamp'].apply(lambda x: x.isoformat())
    df_to_save.to_csv(RATINGS_FILE, index=False)
    print(f"Ratings salvos em {RATINGS_FILE}.")

def save_preferences_to_json_notebook():
    global user_initial_preferences
    serializable_prefs = {str(k): {'name': v.get('name', ''), 
                                   'categories': v.get('categories', []), 
                                   'location': list(v['location']) if 'location' in v and v['location'] is not None else None} 
                          for k, v in user_initial_preferences.items()}
    with open(PREFERENCES_FILE, 'w') as f:
        json.dump(serializable_prefs, f, indent=4)
    print(f"Preferências de usuário salvas em {PREFERENCES_FILE}.")

# Executa o carregamento e pré-processamento
load_and_preprocess_data_notebook()

## 2.1. Nota sobre o Dataset de Avaliações


No nosso sistema, este dataset é gerado artificialmente na inicialização (se nenhum arquivo de ratings persistente for encontrado) para demonstrar o funcionamento do modelo de filtragem colaborativa. A geração artificial é flexível e pode ser ajustada para simular um grande volume de interações, conforme o requisito do projeto.


Este método é fundamental para identificar padrões latentes nas avaliações dos usuários e prever preferências, mesmo com dados esparsos.

## 3. Recomendação Híbrida

Nosso sistema emprega uma abordagem híbrida que combina o melhor de dois mundos:

### 3.1. Filtro Colaborativo (CF) com SVD

O Filtro Colaborativo, especificamente utilizando o algoritmo Singular Value Decomposition (SVD) da biblioteca `Surprise`, baseia-se nas interações (avaliações) dos usuários com os produtos. Ele busca padrões e semelhanças entre usuários ou produtos para prever a preferência de um usuário por um item que ele ainda não avaliou. Se usuários A e B têm gostos similares e o usuário A gostou do produto X, é provável que o usuário B também goste do produto X.

A função `predict_cf_score` usa o modelo SVD treinado para prever a avaliação de um usuário para um produto. O score é normalizado para ficar entre 0 e 1.

In [ ]:
def predict_cf_score(user_id, product_name, model, min_rating=1, max_rating=5):
    """Prevê o score de um item para um usuário usando o modelo SVD."""
    if model is None: 
        return 0.5 # Retorna score neutro se modelo não existe (e.g., sem ratings suficientes)
    
    prediction = model.predict(user_id, product_name, verbose=False)
    normalized_score = (prediction.est - min_rating) / (max_rating - min_rating)
    return normalized_score

# Exemplo de uso:
if svd_model:
    sample_user_id = 1
    sample_product = random.choice(all_available_products) # Escolhe um produto aleatório
    print(f"Predição CF para o Usuário {sample_user_id} e Produto '{sample_product}': {predict_cf_score(sample_user_id, sample_product, svd_model):.2f}")
else:
    print("Modelo SVD não disponível para predição. Certifique-se de que ratings foram simulados/carregados.")

### 3.2. Recomendação Baseada em Conteúdo (CB) com TF-IDF e Similaridade de Cosseno

O Filtro Baseado em Conteúdo foca nas características dos produtos e nas preferências do usuário por essas características. Se um usuário gostou de um Abacate (cremoso, sabor suave), o sistema pode recomendar Graviola (também cremosa, sabor suave) porque compartilha características semelhantes.

Para isso, utilizamos:

* **TF-IDF (`TfidfVectorizer`)**: Transforma as `Combined_Features` de cada produto em uma representação numérica que captura a importância de cada termo (característica) dentro do conjunto de todos os produtos.
* **Similaridade de Cosseno (`cosine_similarity`)**: Mede a similaridade entre os vetores TF-IDF dos produtos. Quanto mais próximos os vetores, mais similares são os produtos em termos de suas características.

A função `predict_cb_score` calcula o score baseado em conteúdo para um produto alvo, comparando-o com os produtos que o usuário avaliou positivamente. Quanto mais similar o produto alvo for aos produtos preferidos do usuário, maior será o score.

In [ ]:
def get_content_similarity(product1_name, product2_name):
    """Calcula a similaridade de cosseno entre dois produtos."""
    if cosine_sim_matrix is None or product1_name not in product_to_idx or product2_name not in product_to_idx:
        return 0.0
    idx1 = product_to_idx[product1_name]
    idx2 = product_to_idx[product2_name]
    return cosine_sim_matrix[idx1, idx2]

def predict_cb_score(user_id, target_product_name, threshold_good_rating=3.5):
    """Prevê o score de um item para um usuário baseado em conteúdo."""
    if cosine_sim_matrix is None or df_ratings_global.empty or not product_to_idx: 
        return 0.0

    user_liked_items_df = df_ratings_global[
        (df_ratings_global['user_id'] == user_id) & 
        (df_ratings_global['rating'] >= threshold_good_rating)
    ]
    if user_liked_items_df.empty: 
        return 0.0

    user_liked_items = user_liked_items_df['item_id'].tolist()

    total_similarity = 0
    count_similar = 0
    for liked_item in user_liked_items:
        similarity = get_content_similarity(target_product_name, liked_item)
        total_similarity += similarity
        count_similar += 1
    return (total_similarity / count_similar) if count_similar > 0 else 0.0

# Exemplo de uso:
if cosine_sim_matrix is not None and not df_ratings_global.empty:
    sample_user_id = 1
    sample_product = random.choice(all_available_products) 
    print(f"Predição CB para o Usuário {sample_user_id} e Produto '{sample_product}': {predict_cb_score(sample_user_id, sample_product):.2f}")
else:
    print("Matriz de similaridade de Cosseno ou ratings não disponíveis para predição CB.")

### 3.3. Combinação dos Modelos e Geração de Recomendações Personalizadas

A essência do nosso sistema híbrido reside na função `combine_scores` e `generate_personalized_recommendations`. Os scores do Filtro Colaborativo (CF) e do Baseado em Conteúdo (CB) são ponderados para gerar um `hybrid_score` final. Isso permite que o sistema aproveite tanto os padrões de comportamento de outros usuários quanto as preferências do usuário pelas características dos produtos.

Além disso, um pequeno "boost" é aplicado ao `hybrid_score` se o produto pertencer a uma categoria que o usuário indicou como preferida em suas preferências iniciais. Isso ajuda a alinhar as recomendações com os interesses explícitos do usuário.

In [ ]:
def combine_scores(cf_score, cb_score, alpha=0.6, beta=0.4):
    """Combina os scores de CF e CB usando pesos."""
    return alpha * cf_score + beta * cb_score

def generate_personalized_recommendations(user_id, alpha=0.6, beta=0.4, top_n=10):
    """Gera recomendações personalizadas para um usuário, combinando CF e CB."""
    user_rated_items = set()
    if not df_ratings_global.empty and user_id in df_ratings_global['user_id'].unique():
        user_rated_items = set(df_ratings_global[df_ratings_global['user_id'] == user_id]['item_id'])
    
    products_to_score = [p for p in all_available_products if p not in user_rated_items]
    if not products_to_score: 
        print("Usuário já avaliou todos os produtos disponíveis ou não há produtos para recomendar.")
        return pd.DataFrame()

    recommendations = []
    for product_name in products_to_score:
        cf_score = predict_cf_score(user_id, product_name, svd_model)
        cb_score = predict_cb_score(user_id, product_name) 
        
        category_boost = 0.0
        if user_id in user_initial_preferences and 'categories' in user_initial_preferences[user_id]:
            prod_cat = df_produtos_features.loc[product_name, 'Categoria_Produto'] if product_name in df_produtos_features.index and 'Categoria_Produto' in df_produtos_features.columns else None
            if prod_cat and prod_cat in user_initial_preferences[user_id]['categories']:
                category_boost = 0.1 

        hybrid_score = combine_scores(cf_score, cb_score, alpha, beta) + category_boost
        recommendations.append({
            'ProductName': product_name,
            'RelevanceScore': hybrid_score,
            'cf_score': cf_score, 
            'cb_score': cb_score  
        })
    
    if not recommendations: return pd.DataFrame()
    recs_df = pd.DataFrame(recommendations)
    return recs_df.sort_values(by='RelevanceScore', ascending=False).head(top_n)

# Exemplo de preferências iniciais para um usuário (simulação para o notebook)
user_initial_preferences[1] = {'categories': ['Fruta', 'Verdura'], 'location': (-15.793889, -47.882778)}

print("Gerando recomendações personalizadas para o usuário 1...")
personalized_recs = generate_personalized_recommendations(user_id=1, top_n=5)
if not personalized_recs.empty:
    print("Recomendações Personalizadas:")
    print(personalized_recs[['ProductName', 'RelevanceScore']])
else:
    print("Não foi possível gerar recomendações personalizadas. Avalie mais produtos ou ajuste suas preferências iniciais.")

### 3.4. Popularidade como Fallback

Para novos usuários que ainda não realizaram avaliações suficientes para que o modelo híbrido gere recomendações personalizadas, ou em cenários onde as recomendações personalizadas não são viáveis, o sistema oferece uma estratégia de fallback: **recomendações baseadas na popularidade**.

A função `get_popular_products_df` identifica os produtos mais populares na plataforma com base no número de avaliações e na média das notas recebidas. Isso garante que o usuário sempre tenha produtos para explorar, mesmo sem um histórico de interações.

In [ ]:
def get_popular_products_df(top_n=10):
    """Retorna os produtos mais populares com base nas avaliações globais."""
    if df_ratings_global.empty or df_ratings_global['item_id'].nunique() < 2: 
        if not all_available_products: return pd.DataFrame()
        print("Não há avaliações suficientes para determinar popularidade. Retornando produtos aleatórios.")
        sample_prods = random.sample(all_available_products, min(top_n, len(all_available_products)))
        return pd.DataFrame([{ 'ProductName': p, 'RelevanceScore': 0.5, 'cf_score':0, 'cb_score':0} for p in sample_prods])

    product_stats = df_ratings_global.groupby('item_id')['rating'].agg(['mean', 'count']).reset_index()
    min_ratings = 2 
    popular = product_stats[product_stats['count'] >= min_ratings]
    popular = popular.sort_values(by=['mean', 'count'], ascending=[False, False]).head(top_n)
    
    print(f"Encontrados {len(popular)} produtos populares.")
    return pd.DataFrame([{
        'ProductName': row['item_id'],
        'RelevanceScore': row['mean'] / 5.0, 
        'cf_score':0, 'cb_score':0 
    } for _, row in popular.iterrows()])

# Exemplo de uso:
popular_recs = get_popular_products_df(top_n=5)
if not popular_recs.empty:
    print("Recomendações Populares:")
    print(popular_recs[['ProductName', 'RelevanceScore']])
else:
    print("Não foi possível gerar recomendações populares.")

## 4. Filtro Geoespacial

A localização é um fator crítico para a agricultura familiar e o acesso a produtos frescos. Nosso sistema integra um filtro geoespacial robusto para garantir que as recomendações sejam relevantes e acessíveis ao usuário.

### 4.1. Cálculo de Distância

A função `calculate_distance_km` utiliza a biblioteca `geopy` e o método `geodesic` para calcular a distância mais curta entre dois pontos na superfície da Terra (levando em conta a curvatura do globo), expressa em quilômetros. Isso é mais preciso do que uma simples distância euclidiana em coordenadas planas.

```python
from geopy.distance import geodesic

def calculate_distance_km(lat1, lon1, lat2, lon2):
    if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
        return float('inf') 
    return geodesic((lat1, lon1), (lat2, lon2)).km
```

### 4.2. Filtragem de Recomendações por Distância

A função `get_final_recommendations_with_coops` é responsável por pegar as recomendações de produtos (geradas pelo modelo híbrido ou de popularidade) e associá-las às cooperativas que os vendem, aplicando um filtro de distância máxima definida pelo usuário. Ela também adiciona detalhes importantes como preço, certificações (Orgânico, Agricultura Familiar) e, crucialmente, uma **razão legível para a recomendação**.

In [ ]:
from recommender_logic import calculate_distance_km


def get_recommendation_reason(user_id, product_name, cf_score, cb_score, like_rating_threshold=4):
    """Gera uma razão humana para a recomendação com base nos scores CF e CB e preferências."""
    reasons = []

    if cb_score > 0.1: 
        user_highly_rated_products_df = df_ratings_global[
            (df_ratings_global['user_id'] == user_id) & (df_ratings_global['rating'] >= like_rating_threshold)
        ]
        if not user_highly_rated_products_df.empty:
            user_highly_rated_products = user_highly_rated_products_df['item_id'].tolist()
            best_match_similarity = 0
            best_match_product = None

            if product_name in product_to_idx:
                for liked_prod in user_highly_rated_products:
                    if liked_prod in product_to_idx:
                        sim = get_content_similarity(product_name, liked_prod)
                        if sim > best_match_similarity:
                            best_match_similarity = sim
                            best_match_product = liked_prod
                
                if best_match_product and best_match_similarity > 0.2:
                    try:
                        target_categoria = df_produtos_features.loc[product_name, 'Categoria_Produto'] if product_name in df_produtos_features.index and 'Categoria_Produto' in df_produtos_features.columns else ""
                        liked_categoria = df_produtos_features.loc[best_match_product, 'Categoria_Produto'] if best_match_product in df_produtos_features.index and 'Categoria_Produto' in df_produtos_features.columns else ""

                        if target_categoria and target_categoria == liked_categoria:
                             reasons.append(f"Similar ao '{best_match_product}' (que você avaliou bem), ambos da categoria '{target_categoria}'.")
                        else:
                             reasons.append(f"Similar ao '{best_match_product}' (que você avaliou bem), com base em suas características gerais.")
                    except Exception as e:
                        reasons.append(f"Similar ao '{best_match_product}' (que você avaliou bem).")
                elif not best_match_product and user_highly_rated_products:
                     reasons.append("Baseado nas características de produtos que você gostou.")

    if cf_score > 0.7 and cb_score < 0.5 :
        reasons.append("Outros usuários com gostos parecidos com os seus também apreciaram este produto.")
    elif cf_score > 0.5 and not reasons: 
         reasons.append("Pode ser do seu interesse com base nas avaliações de outros usuários.")

    if user_id in user_initial_preferences and 'categories' in user_initial_preferences[user_id]:
        prod_cat = df_produtos_features.loc[product_name, 'Categoria_Produto'] if product_name in df_produtos_features.index and 'Categoria_Produto' in df_produtos_features.columns else None
        if prod_cat and prod_cat in user_initial_preferences[user_id]['categories']:
            if not reasons or "você demonstrou interesse na categoria" not in reasons[0]:
                reasons.append(f"Recomendado porque você demonstrou interesse na categoria '{prod_cat}'.")

    if not reasons:
        return "Recomendado para você explorar novos sabores e produtores locais!"
    return " ".join(reasons)

def get_final_recommendations_with_coops(user_id, user_lat, user_lon, max_dist_km, recommendations_df, recommendation_type="personalized"):
    """Junta recomendações de produtos com informações das cooperativas e aplica filtros de distância."""
    if recommendations_df.empty:
        return pd.DataFrame()

    output_recs = []
    
    for _, rec_product_row in recommendations_df.iterrows():
        product_name = rec_product_row['ProductName']
        
        coops_selling_this_product = df_cooperativas[df_cooperativas['Nome_Produto'] == product_name].copy()
        if coops_selling_this_product.empty:
            continue

        coops_selling_this_product['Distance_km'] = coops_selling_this_product.apply(
            lambda r: calculate_distance_km(user_lat, user_lon, r['Latitude'], r['Longitude']), axis=1
        )
        
        coops_within_dist = coops_selling_this_product[coops_selling_this_product['Distance_km'] <= max_dist_km]
        if coops_within_dist.empty:
            continue

        for _, coop_row in coops_within_dist.sort_values(by='Distance_km').iterrows():
            if recommendation_type == "popular":
                reason = "Um dos produtos mais populares da plataforma!"
            elif recommendation_type == "popular_fallback":
                 reason = "Este é um produto popular, pois não conseguimos gerar recomendações personalizadas ainda."
            else: 
                reason = get_recommendation_reason(user_id, product_name, rec_product_row['cf_score'], rec_product_row['cb_score'])
            
            output_recs.append({
                'ProductName': coop_row['Nome_Produto'], 
                'CooperativeName': coop_row['Nome_Cooperativa'], 
                'Region': coop_row['Regioes_Entrega'], 
                'Latitude': coop_row['Latitude'],
                'Longitude': coop_row['Longitude'],
                'Distance_km': coop_row['Distance_km'], 
                'Price': coop_row['Preco'], 
                'Organic': coop_row['Certificacao_Organica_Geral'] >= 1, 
                'FamilyFarm': coop_row['Selo_Agricultura_Familiar_Possui'] == 1, 
                'Horario_Funcionamento_Atendimento': coop_row['Horario_Funcionamento_Atendimento'],
                'Formas_Pagamento_Aceitas': coop_row['Formas_Pagamento_Aceitas'],
                'Faz_Entrega': coop_row['Faz_Entrega'] == 1,
                'RelevanceScore': rec_product_row['RelevanceScore'],
                'Reason': reason 
            })

    if not output_recs: return pd.DataFrame()
    
    final_df = pd.DataFrame(output_recs)
    final_df.sort_values(by=['ProductName', 'RelevanceScore', 'Distance_km'], ascending=[True, False, True], inplace=True) 
    
    return final_df


# Exemplo de uso:
user_lat_ex = -15.80
user_lon_ex = -47.90
max_dist_ex = 20 

print("\n--- Exemplo de Recomendações Finais com Coops e Filtro Geoespacial ---")
if not personalized_recs.empty:
    final_recs = get_final_recommendations_with_coops(
        user_id=1, user_lat=user_lat_ex, user_lon=user_lon_ex, 
        max_dist_km=max_dist_ex, recommendations_df=personalized_recs,
        recommendation_type="personalized"
    ).head(3) 
    if not final_recs.empty:
        print("Recomendações finais para Usuário 1 (próximas de você):")
        print(final_recs[['ProductName', 'CooperativeName', 'Distance_km', 'Price', 'Reason']])
    else:
        print("Nenhuma recomendação personalizada encontrada dentro da distância especificada.")
else:
    print("Não foi possível gerar recomendações personalizadas para testar o filtro geoespacial.")

print("\n--- Exemplo de Busca Direta de Produtos com Filtro Geoespacial ---")
search_results = search_products_in_cooperatives(
    user_lat=user_lat_ex, user_lon=user_lon_ex, max_dist_km=max_dist_ex,
    product_intent="Tomate", preferred_categories=["Legume (culinário)/Fruta (botânico)"]
).head(3)

if not search_results.empty:
    print("Resultados da busca por 'Tomate' (próximas de você):")
    print(search_results[['ProductName', 'CooperativeName', 'Distance_km', 'Price']])
else:
    print("Nenhum 'Tomate' encontrado dentro da distância ou categoria especificada.")

## 5. Lógica de Avaliação do Usuário

A capacidade de um usuário avaliar produtos é fundamental para o sucesso do filtro colaborativo. Cada avaliação fornecida pelo usuário contribui para o seu perfil de gostos e ajuda a refinar as futuras recomendações.

A função `add_rating_logic` é responsável por:

1.  Adicionar ou atualizar uma avaliação de um produto por um usuário específico em uma dada cooperativa.
2.  **Retreinar o modelo SVD**: Após cada nova avaliação (ou atualização de uma existente), o modelo SVD é retreinado com os dados mais recentes. Em um ambiente de produção real, isso seria feito de forma assíncrona ou em lotes para evitar sobrecarga do sistema, mas para fins de demonstração, o retreinamento imediato mostra a adaptabilidade do modelo.

Além disso, a função `get_user_ratings_df` permite recuperar todas as avaliações que um usuário já fez, para que ele possa visualizá-las e gerenciá-las.

In [ ]:
def add_rating_logic_notebook(user_id, product_name, cooperative_name, rating_value):
    global df_ratings_global, svd_model
    print(f"Adicionando/Atualizando avaliação para Usuário {user_id}, Produto '{product_name}', Coop '{cooperative_name}', Nota {rating_value}...")
    df_ratings_global = df_ratings_global[~((df_ratings_global['user_id'] == user_id) & 
                                            (df_ratings_global['item_id'] == product_name) &
                                            (df_ratings_global['cooperative_name'] == cooperative_name))]
    
    new_rating = pd.DataFrame([{
        'user_id': user_id,
        'item_id': product_name,
        'cooperative_name': cooperative_name,
        'rating': int(rating_value),
        'timestamp': datetime.now()
    }])
    df_ratings_global = pd.concat([df_ratings_global, new_rating], ignore_index=True)
    
    print(f"Total de ratings agora: {len(df_ratings_global)}")
    
    save_ratings_to_csv_notebook() # Salva no notebook também

    if not df_ratings_global.empty:
        reader = Reader(rating_scale=(1, 5))
        data = Dataset.load_from_df(df_ratings_global[['user_id', 'item_id', 'rating']], reader)
        trainset = data.build_full_trainset()
        svd_model = SVD(n_factors=50, n_epochs=20, random_state=42, biased=True, verbose=False)
        svd_model.fit(trainset)
        print("Modelo SVD retreinado após novo rating.")
    else:
        svd_model = None
        print("Modelo SVD desativado (sem ratings).")

def get_user_ratings_df_notebook(user_id):
    """Retorna um DataFrame com as avaliações de um usuário específico."""
    if df_ratings_global.empty:
        return pd.DataFrame()
    user_r = df_ratings_global[df_ratings_global['user_id'] == user_id].copy()
    if not user_r.empty:
        user_r['formatted_timestamp'] = user_r['timestamp'].dt.strftime('%d/%m/%Y %H:%M')
        user_r.rename(columns={'item_id': 'product_name'}, inplace=True)
    return user_r.sort_values(by='timestamp', ascending=False)


# Exemplo de uso:
print("\n--- Exemplo de Avaliação e Retreinamento ---")
test_user_id = 101 # Um novo usuário de teste
test_product_1 = "Manga"
test_coop_1 = df_cooperativas[df_cooperativas['Nome_Produto'] == test_product_1]['Nome_Cooperativa'].iloc[0] if not df_cooperativas[df_cooperativas['Nome_Produto'] == test_product_1].empty else "Cooperativa Teste"
test_product_2 = "Banana"
test_coop_2 = df_cooperativas[df_cooperativas['Nome_Produto'] == test_product_2]['Nome_Cooperativa'].iloc[0] if not df_cooperativas[df_cooperativas['Nome_Produto'] == test_product_2].empty else "Cooperativa Teste"

add_rating_logic_notebook(test_user_id, test_product_1, test_coop_1, 5)
add_rating_logic_notebook(test_user_id, test_product_2, test_coop_2, 4)

print("\nAvaliações do Usuário Teste:")
print(get_user_ratings_df_notebook(test_user_id)[['product_name', 'cooperative_name', 'rating', 'formatted_timestamp']])

## 6. Conclusão

Este sistema de recomendação agrícola oferece uma solução abrangente para conectar produtores e consumidores, utilizando técnicas avançadas de Inteligência Artificial:

* **Hibridização (CF + CB)**: Combina a força das avaliações dos usuários com as características dos produtos para recomendações mais precisas e personalizadas.
* **Sensibilidade Geoespacial**: Garante que as recomendações sejam não apenas relevantes, mas também acessíveis fisicamente ao consumidor.
* **Interface Amigável**: A integração com Flask e Leaflet.js proporciona uma experiência de usuário intuitiva e visual.
* **Relevância Local**: Utiliza dados de produtores e produtos do Distrito Federal e Entorno, o que é um diferencial e um extra do projeto.

O projeto está estruturado de forma modular, permitindo futuras expansões e aprimoramentos, como a incorporação de sazonalidade, mais tipos de produtos ou filtros de preferência para certificações específicas (Orgânico, Agricultura Familiar).